In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# from datasets import load_dataset
# dataset = load_dataset("achrafothman/aslg_pc12")
# print(dataset)

glosses = [
    "Hello Me English Name D I C K",
    "Happy Meet You",
]
texts = [
    "Hello, my English name is Dick.",
    "Nice to meet you."
]

from datasets import Dataset, DatasetDict
import pandas as pd
df = pd.DataFrame({
    "gloss": glosses,
    "text": texts
})

train_dataset = Dataset.from_pandas(df)
dataset = DatasetDict({
    "train": train_dataset,
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['gloss', 'text'],
        num_rows: 2
    })
})


In [3]:
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments

tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    inputs = examples['gloss']
    targets = examples['text']
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Model
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
model.config.pad_token_id = tokenizer.eos_token_id

c:\Users\USER\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

c:\Users\USER\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [4]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=32,
    weight_decay=0.01,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['train'],
)

# Train the model
trainer.train()

c:\Users\USER\anaconda3\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/32 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.3673299252986908, 'eval_runtime': 0.5569, 'eval_samples_per_second': 3.591, 'eval_steps_per_second': 1.796, 'epoch': 1.0}


  0%|          | 0/1 [00:00<?, ?it/s]

...

{'eval_loss': 0.002621282823383808, 'eval_runtime': 0.5144, 'eval_samples_per_second': 3.888, 'eval_steps_per_second': 1.944, 'epoch': 31.0}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warni

  0%|          | 0/1 [00:00<?, ?it/s]

{'eval_loss': 0.002547711133956909, 'eval_runtime': 9.6435, 'eval_samples_per_second': 0.207, 'eval_steps_per_second': 0.104, 'epoch': 32.0}
{'train_runtime': 133.0056, 'train_samples_per_second': 0.481, 'train_steps_per_second': 0.241, 'train_loss': 0.15988802909851074, 'epoch': 32.0}


TrainOutput(global_step=32, training_loss=0.15988802909851074, metrics={'train_runtime': 133.0056, 'train_samples_per_second': 0.481, 'train_steps_per_second': 0.241, 'total_flos': 4877891665920.0, 'train_loss': 0.15988802909851074, 'epoch': 32.0})

In [5]:
# save the model and the tokenizer
model.save_pretrained("./llm_model")
tokenizer.save_pretrained("./llm_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
Your generation config was originally created from the model config, but the model config has changed since then. Unless you pass the `generation_config` argument to this model's `generate` calls, they will revert to the legacy behavior where the base `generate` parameterization is loaded from the model config instead. To avoid this behavior and this warning, we recommend you to overwrite the generation config model attribute before calling the model's `save_pretrained`, preferably also removing any generation kwargs from the model config. This warni

('./llm_model\\tokenizer_config.json',
 './llm_model\\special_tokens_map.json',
 './llm_model\\vocab.json',
 './llm_model\\merges.txt',
 './llm_model\\added_tokens.json')

In [6]:
# model prediction using training set
import torch

def predict(input_text):
    inputs = tokenizer(input_text, max_length=128, return_tensors='pt', truncation=True, padding='max_length')
    with torch.no_grad():
        pred = model.generate(**inputs)
    return tokenizer.decode(pred[0], skip_special_tokens=True)

for i in range(len(glosses)):
    print("Gloss:", glosses[i])
    print("Prediction:", predict(glosses[i]))
    print("Target:", texts[i])
    print()

Gloss: Hello Me English Name D I C K


c:\Users\USER\anaconda3\Lib\site-packages\transformers\generation\utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
c:\Users\USER\anaconda3\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Prediction: Hello, my English name is Dick.
Target: Hello, my English name is Dick.

Gloss: Happy Meet You
Prediction: Nice to meet you.
Target: Nice to meet you.



In [7]:
# load model
model = BartForConditionalGeneration.from_pretrained("./llm_model")
tokenizer = BartTokenizer.from_pretrained("./llm_model")

# model prediction using training set after loading
for i in range(len(glosses)):
    print("Gloss:", glosses[i])
    print("Prediction:", predict(glosses[i]))
    print("Target:", texts[i])
    print()

Gloss: Hello Me English Name D I C K
Prediction: Hello, my English name is Dick.
Target: Hello, my English name is Dick.

Gloss: Happy Meet You
Prediction: Nice to meet you.
Target: Nice to meet you.

